Install Packages


In [1]:
!pip install -q spacy
!pip install -q spacy-lookups-data

!pip install benepar

!pip install -q transformers

!pip install huggingface_hub


!pip install pydot
!pip install evaluate



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14

In [2]:
import spacy, benepar
!python -m spacy download en_core_web_md

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

from transformers import BertTokenizer, TFBertModel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 14.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
#Defines the Models
spacy_checkpoint = "en_core_web_md"

bert_checkpoint = 'bert-base-cased'

benepar.download('benepar_en3')

[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Unzipping models/benepar_en3.zip.


True

In [4]:
nlp = spacy.load(spacy_checkpoint)

if spacy.__version__.startswith('2'):
  nlp.add_pipe(benepar.BeneparComponent("benepar_en3"))
else:
  nlp.add_pipe("benepar", config={"model": "benepar_en3"})


bert_tokenizer = BertTokenizer.from_pretrained(bert_checkpoint)
bert_model = TFBertModel.from_pretrained(bert_checkpoint)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [5]:
# val_df = pd.read_csv('cloze_train_winter2018.csv')
# test_df = pd.read_csv('cloze_test_winter2018.csv')
train_df = pd.read_csv('ROCStories_winter2017.csv')

Modify Training Dataset (Geronimo's Shuffle)
1. create random incorrect ending
2. shuffle position of correct and incorrect ending
3. define lables based on shuffled positions

In [6]:
# function to randomly shuffle each row across the correct and incorrect ending columns
def shuffle_row(row):
    values = row.values
    np.random.shuffle(values)
    return pd.Series(values, index=row.index)

In [7]:
train_df['WrongSentence'] = train_df['sentence5'].sample(frac=1,replace=False).values + 'INCORRECT_ENDING'
train_df.iloc[:,-2:] = train_df.iloc[:,-2:].apply(shuffle_row,axis=1)

train_df['label'] = train_df.apply(lambda row: 1 if 'INCORRECT_ENDING' in row['sentence5'] else (0 if 'INCORRECT_ENDING' in row['WrongSentence'] else None), axis=1)
train_df.iloc[:,-3:-1] = train_df.iloc[:,-3:-1].applymap(lambda row: row.replace('INCORRECT_ENDING', ''))

train_df = train_df.rename(columns={'sentence5':'ending0','WrongSentence':'ending1'})

# Approach Undecided: Will pursue combining all the sentences first. Will combine endings into column of choices.

In [8]:
def preprocess(df):
    story = " ".join([df['sentence1'], df['sentence2'], df['sentence3'], df['sentence4']])
    choices = [df['ending0'], df['ending1']]
    return story, choices

In [9]:
train_df['story'], train_df['choices'] = zip(*train_df.apply(preprocess, axis=1))
train_df

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,ending0,ending1,label,story,choices
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet...",We were still able to get a good night's rest.,0,David noticed he had put on a lot of weight re...,"[After a few weeks, he started to feel much be..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...,"After that, she wasn't that afraid of little m...",0,Tom had a very short temper. One day a guest m...,[Tom sat on his couch filled with regret about...
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,I thought that was silly.,Marcus was happy to have the right clothes for...,1,Marcus needed clothing for a business casual e...,"[I thought that was silly., Marcus was happy t..."
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...,Kate was beginning to regret her decision to j...,0,Bobby thought Bill should buy a trailer and ha...,[He ended up buying the truck he wanted despit...
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.,They didn't know she knew how to cook.,0,John was a pastor with a very bad memory. He t...,[His congregation was delighted and so was he....
...,...,...,...,...,...,...,...,...,...,...,...
52660,134e8636-3617-43d8-ba6a-9a11b3b115b1,Flavor,The man liked the flavor.,He tried to recreate it at home.,He could not get the flavor right.,He asked the owner of the recipe for help.,So he got kicked out of the game.,The owner of the flavor sold him the recipe.,1,The man liked the flavor. He tried to recreate...,"[So he got kicked out of the game., The owner ..."
52661,4c317f76-ca42-4024-a4c2-12ec911cf89b,After Death,"After my friend's dad's funeral, I got in trou...",The principal said I wasn't allowed to leave s...,He found out I had my friend sign me out.,He told me I was getting detention.,I skipped detention all week.,She surmised the newlyweds with it a week afte...,0,"After my friend's dad's funeral, I got in trou...","[I skipped detention all week., She surmised t..."
52662,a18fd0d2-4d0c-4316-befe-e3d827fe699b,Janice breaks her wrist,Janice was out exercising for her big soccer g...,She was doing some drills with her legs.,While working out and exercising she slips on ...,She falls down and uses her wrist to break her...,Tom decided to get a different kind.,She breaks her wrist in the process and goes t...,1,Janice was out exercising for her big soccer g...,"[Tom decided to get a different kind., She bre..."
52663,2c14252b-4080-4fca-8765-537772018508,Jamie marries for love,Jamie is an american girl.,Jamie wants to get married to a mexican man.,Her family assumes it's because the man wants ...,Jamie insist that she is marrying him out of l...,"The next week, he did the same thing.",Jamie gets married and they spent the rest of ...,1,Jamie is an american girl. Jamie wants to get ...,"[The next week, he did the same thing., Jamie .

In [10]:
train_df[:5]

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,ending0,ending1,label,story,choices
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet...",We were still able to get a good night's rest.,0,David noticed he had put on a lot of weight re...,"[After a few weeks, he started to feel much be..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...,"After that, she wasn't that afraid of little m...",0,Tom had a very short temper. One day a guest m...,[Tom sat on his couch filled with regret about...
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,I thought that was silly.,Marcus was happy to have the right clothes for...,1,Marcus needed clothing for a business casual e...,"[I thought that was silly., Marcus was happy t..."
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...,Kate was beginning to regret her decision to j...,0,Bobby thought Bill should buy a trailer and ha...,[He ended up buying the truck he wanted despit...
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.,They didn't know she knew how to cook.,0,John was a pastor with a very bad memory. He t...,[His congregation was delighted and so was he....


In [11]:
# To Help Visualize Dependency Tree
import nltk
from nltk import Tree, ParentedTree

from spacy import displacy

In [12]:
# Example of Parser Working on Data
doc = nlp(train_df.loc[0,'story'])
# print (doc)
sent = list(doc.sents)[0]
x = sent._.parse_string
print (x)
parse_tree = ParentedTree.fromstring('(' + sent._.parse_string + ')')
print(parse_tree.pretty_print())
print (sent._.parse_string)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/distributions/distribution.py:53: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


(S (NP (NNP David)) (VP (VBD noticed) (SBAR (S (NP (PRP he)) (VP (VBD had) (VP (VBN put) (PRT (RP on)) (NP (NP (DT a) (NN lot)) (PP (IN of) (NP (NN weight)))) (ADVP (RB recently))))))) (. .))
                                                                          
                            |                                              
                            S                                             
   _________________________|___________________________________________   
  |            VP                                                       | 
  |       _____|_______                                                 |  
  |      |            SBAR                                              | 
  |      |             |                                                |  
  |      |             S                                                | 
  |      |      _______|________                                        |  
  |      |     |                VP                   

In [13]:
displacy.render(doc, style='dep')

In [14]:
#Code to Create Dependency
def create_dependency(df):
    df['dependency_tree'] = None
    for index, row in df.iterrows():
        doc = nlp(row['story'])
        dependency = [sent._.parse_string for sent in doc.sents]
        df.at[index, 'dependency_tree'] = dependency
    return df


In [15]:
subset = train_df[:10]
subset = create_dependency(subset)

<ipython-input-14-a8289260f69d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dependency_tree'] = None
/usr/local/lib/python3.10/dist-packages/torch/distributions/distribution.py:53: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


In [16]:
# train_df = train_df.drop(['dependency_tree'], axis=1)
subset.loc[0,'dependency_tree']

['(S (NP (NNP David)) (VP (VBD noticed) (SBAR (S (NP (PRP he)) (VP (VBD had) (VP (VBN put) (PRT (RP on)) (NP (NP (DT a) (NN lot)) (PP (IN of) (NP (NN weight)))) (ADVP (RB recently))))))) (. .))',
 '(S (NP (PRP He)) (VP (VBD examined) (NP (PRP$ his) (NNS habits)) (S (VP (TO to) (VP (VP (VB try)) (CC and) (VP (VB figure) (PRT (RP out)) (NP (DT the) (NN reason))))))) (. .))',
 "(S (NP (PRP He)) (VP (VBD realized) (SBAR (S (NP (PRP he)) (VP (VBD 'd) (VP (VBN been) (VP (VBG eating) (NP (ADJP (RB too) (JJ much)) (JJ fast) (NN food)) (ADVP (RB lately)))))))) (. .))",
 '(S (NP (PRP He)) (VP (VP (VBD stopped) (S (VP (VBG going) (PP (IN to) (NP (NN burger) (NNS places)))))) (CC and) (VP (VBD started) (NP (DT a) (JJ vegetarian) (NN diet)))) (. .))']

In [17]:
# def tokenize_story_choices(row):
#     story_tokens = tokenizer.encode_plus(row['story'], add_special_tokens=True, max_length=512, truncation=True, padding='max_length', return_tensors='tf')
#     choice_tokens = tokenizer.encode_plus(row['choices'], add_special_tokens=False, max_length=128, truncation=True, padding='max_length', return_tensors='tf')
#     return {
#         'input_word_ids': story_tokens['input_ids'][0],
#         'input_mask': story_tokens['attention_mask'][0],
#         'segment_ids': story_tokens['token_type_ids'][0]
#     }, {
#         'input_word_ids': choice_tokens['input_ids'],
#         'input_mask': choice_tokens['attention_mask'],
#         'segment_ids': choice_tokens['token_type_ids']
#     }

In [18]:
import torch
import spacy
import benepar
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

# Initialize spaCy and benepar
nlp = spacy.load("en_core_web_sm")

if spacy.__version__.startswith('2'):
    nlp.add_pipe(benepar.BeneparComponent("benepar_en3"))
else:
    nlp.add_pipe("benepar", config={"model": "benepar_en3"})

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Example data
data = {
    "storytitle": "David Drops the Weight",
    "sentences": [
        "David noticed he had put on a lot of weight recently.",
        "He examined his habits to try and figure out the reason.",
        "He realized he'd been eating too much fast food lately.",
        "He stopped going to burger places and started a vegetarian diet.",
        "After a few weeks, he started to feel much better.",
        "Tom sat on his couch filled with regret about his actions."  # This is the incorrect ending
    ]
}

# Function to extract triples from a sentence using spaCy
def extract_triples(sentence):
    doc = nlp(sentence)
    triples = []
    subject = ""
    predicate = ""
    object_ = ""
    for token in doc:
        # print (token,token.dep_)
        # print("token.dep")
        # print("\n")
        if token.dep_ == 'ROOT':
          predicate += token.text
        for child in token.children:
            # print ('---------------------')
            # print(subject,predicate,object_)
            # print("token:",token)
            # print ("child:",child)
            # print ("child.dep:",child.dep_)
            # print ('---------------------')
            if child.dep_ == 'nsubj' and len(subject) == 0:
                subject += child.text
                subject += " "
            elif child.dep_ == 'dobj' or child.dep_ == 'pobj':
                object_ += child.text
                object_ += " "
            # elif child.dep_ == 'nsubjpass':
            #     object_ = child.text
            # elif child.dep_ == 'aux' or child.dep_ == 'auxpass':
            #     predicate += child.text + " "
            # elif child.dep_ == 'ccomp':
            #     predicate += child.text + " "
            #     for grandchild in child.children:
            #         if grandchild.dep_ == 'dobj':
            #             object_ = grandchild.text
            # print ("\n")
    triples.append((subject.strip(), predicate.strip(), object_.strip()))
    # print("tripled:",triples)
        # print("subjec:",subject,"pred:",predicate,"onjcet:",object_)
    return triples

# Extract triples from sentences
triples = []
for sentence in data["sentences"]:
    triples.extend(extract_triples(sentence))
    print ('new sentence \n')



/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


new sentence 

new sentence 

new sentence 

new sentence 

new sentence 

new sentence 



In [29]:
# Tokenize and encode triples
encoded_triples = []
for triple in triples[0:4]:
    encoded_triple = tokenizer(triple[0], triple[1], triple[2], return_tensors='pt', padding=True, truncation=True)
    encoded_triples.append(encoded_triple)

# Pass encoded triples through BERT
with torch.no_grad():
    embeddings = []
    for encoded_triple in encoded_triples:
        input_ids = encoded_triple['input_ids'].to(model.device)
        attention_mask = encoded_triple['attention_mask'].to(model.device)
        token_type_ids = encoded_triple['token_type_ids'].to(model.device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling
        embeddings.append(pooled_output)

        # print(embeddings)

        # print(pooled_output)
        # print('------------------------------------------')
        # print(pooled_output.shape)
        # print(pooled_output.squeeze().shape)

# Compute similarities between triples and the incorrect ending
wrong_ending_similar = []
for i in range(len(embeddings)):
    encoded_sentence = tokenizer(data["sentences"][-1:], return_tensors='pt', padding=True, truncation=True)  # Change here
    with torch.no_grad():
        outputs = model(**encoded_sentence)
        pooled_output = torch.mean(outputs.last_hidden_state, dim=1).squeeze()  # Mean pooling and squeeze
    sim = 1 - cosine(embeddings[i].squeeze(), pooled_output)  # Compare with the last sentence (incorrect ending) and squeeze
    wrong_ending_similar.append((triples[i], sim))

# Compute similarities between triples and the correct ending
right_ending_similar = []
for i in range(len(embeddings)):
    encoded_sentence = tokenizer(data["sentences"][-2], return_tensors='pt', padding=True, truncation=True)  # Change here
    with torch.no_grad():
        outputs = model(**encoded_sentence)
        pooled_output = torch.mean(outputs.last_hidden_state, dim=1).squeeze()  # Mean pooling and squeeze
    sim = 1 - cosine(embeddings[i].squeeze(), pooled_output)  # Compare with the 2nd last sentence (correct ending) and squeeze
    right_ending_similar.append((triples[i], sim))


# Rank triples based on similarities
wrong_ranked_triples = sorted(wrong_ending_similar, key=lambda x: x[1], reverse=True)

right_ranked_triples = sorted(right_ending_similar, key=lambda x: x[1], reverse=True)

# Print Wrong Ending ranked triples
print("Wrong Ranked Triples:")
sim = []
for idx, (triple, sim) in enumerate(wrong_ranked_triples, start=1):
    print(f"Rank {idx}: Similarity {sim:.4f} - {triple}")
print ("Average:", np.average(sim))

# Print Right Ending ranked triples
sim = []
print("\nRight Ranked Triples:")
for idx, (triple, sim) in enumerate(right_ranked_triples, start=1):
    print(f"Rank {idx}: Similarity {sim:.4f} - {triple}")
print ("Average:", np.average(sim))

# Print correct and incorrect endings
print("\nCorrect Ending:", data["sentences"][-2])  # Last sentence (correct ending)
print("Incorrect Ending:", data["sentences"][-1])  # Last sentence (incorrect ending)

Wrong Ranked Triples:
Rank 1: Similarity 0.3516 - ('He', 'stopped', 'places diet')
Rank 2: Similarity 0.3320 - ('He', 'realized', 'food')
Rank 3: Similarity 0.3177 - ('He', 'examined', 'habits reason')
Rank 4: Similarity 0.2858 - ('David', 'noticed', 'lot weight')
Average: 0.2858285903930664

Right Ranked Triples:
Rank 1: Similarity 0.3836 - ('He', 'realized', 'food')
Rank 2: Similarity 0.3767 - ('He', 'stopped', 'places diet')
Rank 3: Similarity 0.3324 - ('He', 'examined', 'habits reason')
Rank 4: Similarity 0.3221 - ('David', 'noticed', 'lot weight')
Average: 0.3221202492713928

Correct Ending: After a few weeks, he started to feel much better.
Incorrect Ending: Tom sat on his couch filled with regret about his actions.


In [20]:
#Create Triple for Story in DF
def df_graph(df):
  dic = {'sentences':[]}
  df['story_triple'] = None
  for index, row in df.iterrows():
    dic['sentences'].append(df.at[index,'sentence1'])
    dic['sentences'].append(df.at[index,'sentence2'])
    dic['sentences'].append(df.at[index,'sentence3'])
    dic['sentences'].append(df.at[index,'sentence4'])

    triples = []

    for sentence in dic['sentences']:
      triples.extend(extract_triples(sentence))
    # print(triples)
    df.at[index,'story_triple'] = triples
    dic = {'sentences':[]}
  return df[["story","story_triple","choices","label"]]

In [21]:
sample_train = df_graph(train_df[0:5])
sample_train

<ipython-input-20-23468e65febf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['story_triple'] = None
/usr/local/lib/python3.10/dist-packages/torch/distributions/distribution.py:53: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


,story,story_triple,choices,label
0,David noticed he had put on a lot of weight re...,"[(David, noticed, lot weight), (He, examined, ...","[After a few weeks, he started to feel much be...",0
1,Tom had a very short temper. One day a guest m...,"[(Tom, had, temper), (guest, made, ), (He, pun...",[Tom sat on his couch filled with regret about...,0
2,Marcus needed clothing for a business casual e...,"[(Marcus, needed, clothing event), (All, were,...","[I thought that was silly., Marcus was happy t...",1
3,Bobby thought Bill should buy a trailer and ha...,"[(Bobby, thought, trailer it car), (Bill, thou...",[He ended up buying the truck he wanted despit...,0
4,John was a pastor with a very bad memory. He t...,"[(John, was, memory), (He, tried, sermons adva...",[His congregation was delighted and so was he....,0


In [22]:
def tokenize_for_cosine(df):
  df['triple_embedding']= None
  df['choices_outputs']= None
  for index, row in df.iterrows():
    encoded_triples = []
    for triple in df.at[index,'story_triple']:
        encoded_triple = tokenizer(triple[0], triple[1], triple[2], return_tensors='pt', padding=True, truncation=True)
        encoded_triples.append(encoded_triple)
    # Pass encoded triples through BERT
    with torch.no_grad():
        embeddings = []
        for encoded_triple in encoded_triples:
            input_ids = encoded_triple['input_ids'].to(model.device)
            attention_mask = encoded_triple['attention_mask'].to(model.device)
            token_type_ids = encoded_triple['token_type_ids'].to(model.device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            pooled_output = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling
            embeddings.append(pooled_output)

        df.at[index,'triple_embedding'] = embeddings

    choice_output = []
    for choice in df.at[index,'choices']:
      encoded_sentence = tokenizer(choice, return_tensors='pt', padding=True, truncation=True)  # Change here
      with torch.no_grad():
        outputs = model(**encoded_sentence)
        pooled_output = torch.mean(outputs.last_hidden_state, dim=1).squeeze()
        choice_output.append(pooled_output)
    df.at[index,'choices_outputs'] = choice_output
  return df

In [23]:
token_sample_train = tokenize_for_cosine(sample_train)
token_sample_train

,story,story_triple,choices,label,triple_embedding,choices_outputs
0,David noticed he had put on a lot of weight re...,"[(David, noticed, lot weight), (He, examined, ...","[After a few weeks, he started to feel much be...",0,"[[[tensor(0.2171), tensor(0.2885), tensor(0.02...","[[tensor(0.0843), tensor(-0.2282), tensor(0.17..."
1,Tom had a very short temper. One day a guest m...,"[(Tom, had, temper), (guest, made, ), (He, pun...",[Tom sat on his couch filled with regret about...,0,"[[[tensor(0.3617), tensor(0.2041), tensor(-0.1...","[[tensor(0.0746), tensor(0.3324), tensor(-0.05..."
2,Marcus needed clothing for a business casual e...,"[(Marcus, needed, clothing event), (All, were,...","[I thought that was silly., Marcus was happy t...",1,"[[[tensor(0.0970), tensor(0.2913), tensor(-0.1...","[[tensor(0.0630), tensor(-0.0570), tensor(-0.1..."
3,Bobby thought Bill should buy a trailer and ha...,"[(Bobby, thought, trailer it car), (Bill, thou...",[He ended up buying the truck he wanted despit...,0,"[[[tensor(0.1580), tensor(0.0592), tensor(0.06...","[[tensor(-0.2134), tensor(-0.3301), tensor(0.0..."
4,John was a pastor with a very bad memory. He t...,"[(John, was, memory), (He, tried, sermons adva...",[His congregation was delighted and so was he....,0,"[[[tensor(0.0170), tensor(-0.0448), tensor(-0....","[[tensor(0.2973), tensor(-0.0552), tensor(0.21..."


In [53]:
def cosine_scoring(df):
  df['average'] = None
  for index, row in df.iterrows():
    ending0_avg = []
    ending1_avg = []
    for x in df.at[index,'triple_embedding']:
      ending0_avg.append(1 - cosine(x.squeeze(), df.at[index,'choices_outputs'][0]))
      ending1_avg.append(1 - cosine(x.squeeze(), df.at[index,'choices_outputs'][1]))
    # print(ending0_avg)
    # print(ending1_avg)
    ending0_avg = np.average(ending0_avg)
    ending1_avg = np.average(ending1_avg)
    df.at[index,'average'] = [ending0_avg,ending1_avg]
  return df

In [34]:
finish_train = cosine_scoring(token_sample_train[:1])

[0.3221202492713928, 0.3324408233165741, 0.3835591971874237, 0.3766587972640991]
[0.2975882291793823, 0.29088833928108215, 0.296907514333725, 0.3378101587295532]


<ipython-input-24-43e7be4eb44f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['average'] = None


In [43]:
def get_accuracy(df):
  total = len(df)
  positive = 0
  for index, row in df.iterrows():
    if np.max(df.at[index,'average']) == df.at[index,'average'][df.at[index,'label']]:
      positive += 1
  negative = total-positive
  accuracy = positive/total
  return accuracy, positive, negative, total

In [56]:
def all_together(df):
  '''This function will package together all the functions created to produce a final result.
  The df must have columns: sentence1,	sentence2,	sentence3,	sentence4, story, choices, and label.
  Not included in here is the creation of story (joint of sent1-sent4) or choices[list of both endings]'''
  df = df_graph(df)
  print("\nDone w/ Graph\n")
  df = tokenize_for_cosine(df)
  print("\nDone w/ Token\n")
  df = cosine_scoring(df)
  print("\nDone w/ Scoring\n")
  final = get_accuracy(df)
  return final

In [57]:
all_together(train_df[:5000])

<ipython-input-20-23468e65febf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['story_triple'] = None



Done w/ Graph



<ipython-input-22-f73c6bd78dcf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['triple_embedding']= None
<ipython-input-22-f73c6bd78dcf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['choices_outputs']= None



Done w/ Token



<ipython-input-53-81fc9ce54d89>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['average'] = None



Done w/ Scoring



(0.501, 501, 499, 1000)